In [1]:
import urllib
import warnings

import optuna
import sklearn
import joblib


import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras as keras
sns.set()

from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Conv1D, Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling1D
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

from utils.utils import read_dataset

In [2]:
BATCHSIZE = 16
EPOCHS = 500

In [4]:
# 範例的模型
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(-1, img_x, img_y, 1)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    input_shape = (img_x, img_y, 1)

    model = Sequential()
    model.add(
        Conv2D(
            filters=trial.suggest_categorical("filters", [32, 64]),
            kernel_size=trial.suggest_categorical("kernel_size", [3, 5]),
            strides=trial.suggest_categorical("strides", [1, 2]),
            activation=trial.suggest_categorical("activation", ["relu", "linear"]),
            input_shape=input_shape,
        )
    )
    model.add(Flatten())
    model.add(Dense(CLASSES, activation="softmax"))

    # We compile our model with a sampled learning rate.
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=RMSprop(learning_rate=learning_rate),
        metrics=["accuracy"],
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

In [77]:
    archive_name = "UCRArchive_2018"
    # dataset_name = 'BME'
    dataset_name = 'ChlorineConcentration'
    root_dir = 'D:/FH_Dortmund/Projekt/Projektarbeit_2/time_series/dl-4-tsc-master/dataset'
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_valid = datasets_dict[dataset_name][2]
    y_valid = datasets_dict[dataset_name][3]
    
    nb_classes = len(np.unique(np.concatenate((y_train, y_valid), axis=0)))
    
    
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train, y_valid), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_valid = enc.transform(y_valid.reshape(-1, 1)).toarray()
    
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)).astype("float32") / 255
    x_valid = x_valid.reshape((x_valid.shape[0], x_valid.shape[1], 1)).astype("float32") / 255
    
 #   y_train_original = y_train
 #   y_train = y_train[:N_TRAIN_EXAMPLES]
 #   y_valid = y_valid[:N_VALID_EXAMPLES]
    # y_train = y_train[:N_TRAIN_EXAMPLES].astype(int)
    # y_valid = y_valid[:N_VALID_EXAMPLES].astype(int)
    
#    y_train = to_categorical(y_train, num_classes=3)
#   y_valid = to_categorical(y_valid, num_classes=3)
    #y_train = to_categorical(dataset['y_train'].values)
    
    
    #CLASSES = len(np.unique(np.concatenate((y_train, y_valid), axis=0)))
    #CLASSES = len((np.concatenate((y_train, y_valid), axis=0)))
    
    classes = np.unique(y_train)
    
#    print(x_train)
#    print(x_valid)
    print("y_train: ", y_train)
#    print("y_valid: ", y_valid)
    print(nb_classes)

#    print(type(CLASSES))
#    print(CLASSES)
    #print(img_x)

y_train:  [[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
3


In [54]:
    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(-1, img_x, img_y, 1)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    input_shape = (img_x, img_y, 1)
    
    print(x_train)
    print(x_valid)
    

[[[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.

In [22]:
# our optuna model
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    # ------------------------------
    # read data
    archive_name = "UCRArchive_2018"
    dataset_name = 'DistalPhalanxOutlineAgeGroup'
    root_dir = 'D:/FH_Dortmund/Projekt/Projektarbeit_2/time_series/dl-4-tsc-master/dataset'
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)
    
    # -------------------------------
    # Input data collation
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_valid = datasets_dict[dataset_name][2]
    y_valid = datasets_dict[dataset_name][3]
    
    # read the number of class
    CLASSES = len(np.unique(np.concatenate((y_train, y_valid), axis=0)))
    
    # data processing
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)).astype("float32") / 255
    x_valid = x_valid.reshape((x_valid.shape[0], x_valid.shape[1], 1)).astype("float32") / 255
    
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train, y_valid), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_valid = enc.transform(y_valid.reshape(-1, 1)).toarray()

    # -------------------------------    
    # start of cnn coding  
    input_shape = x_train.shape[1:] 
    
    # create the model
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(
        Conv1D(
            filters=trial.suggest_categorical("filters_0", [4, 8, 16, 32, 64, 128, 256]),
            kernel_size=trial.suggest_int("kernel_size_0", 3, 15),
            activation = "relu",
            padding="same",
            input_shape=input_shape,
        )
    )
    model.add(BatchNormalization())
    model.add(Dropout(rate=trial.suggest_discrete_uniform('drop_out_0', 0.05, 0.5, 0.05)))
    
    num_layer = trial.suggest_int("num_layer", 1, 3)
    
    for i in range(1, num_layer):
        model.add(
            Conv1D(
                filters=trial.suggest_categorical("filters_"+str(i), [4, 8, 16, 32, 64, 128, 256]), 
                kernel_size=trial.suggest_int("kernel_size_"+str(i), 3, 15), 
                padding="same", 
                activation="relu"))
        model.add(BatchNormalization())
        model.add(Dropout(rate=trial.suggest_discrete_uniform('drop_out_'+str(i), 0.05, 0.5, 0.05)))
        
    model.add(GlobalAveragePooling1D())
    
    model.add(Dropout(rate=trial.suggest_discrete_uniform('drop_out_3', 0.05, 0.5, 0.05)))
    
    model.add(Dense(CLASSES, activation="softmax"))

    tf.keras.callbacks.History()
    
    # We compile our model with a sampled learning rate.
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    model.compile(
        loss="categorical_crossentropy",
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=["accuracy"],
    )
    
    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )


    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

In [23]:
if __name__ == "__main__":
    warnings.warn(
        "Recent Keras release (2.4.0) simply redirects all APIs "
        "in the standalone keras package to point to tf.keras. "
        "There is now only one Keras: tf.keras. "
        "There may be some breaking changes for some workflows by upgrading to keras 2.4.0. "
        "Test before upgrading. "
        "REF:https://github.com/keras-team/keras/releases/tag/2.4.0"
    )
    study = optuna.create_study(direction="maximize")
    joblib.dump(study, "study.pkl")
    #study.optimize(objective, n_trials=100, timeout=600)
    study.optimize(objective, n_trials=100)
    
    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

D:\TMP/ipykernel_14888/813466988.py:2: UserWarning:

Recent Keras release (2.4.0) simply redirects all APIs in the standalone keras package to point to tf.keras. There is now only one Keras: tf.keras. There may be some breaking changes for some workflows by upgrading to keras 2.4.0. Test before upgrading. REF:https://github.com/keras-team/keras/releases/tag/2.4.0

[I 2022-04-02 18:09:32,045] A new study created in memory with name: no-name-c57c39bd-e600-4ce7-877b-dcbf4a6a4166
[I 2022-04-02 18:10:57,322] Trial 0 finished with value: 0.7410072088241577 and parameters: {'num_layer': 2, 'filters_0': 4, 'kernel_size_0': 11, 'filters_1': 16, 'kernel_size_1': 14, 'learning_rate': 0.0012143388202631932}. Best is trial 0 with value: 0.7410072088241577.
[I 2022-04-02 18:12:15,426] Trial 1 finished with value: 0.4676258862018585 and parameters: {'num_layer': 1, 'filters_0': 64, 'kernel_size_0': 9, 'learning_rate': 1.959370729340143e-05}. Best is trial 0 with value: 0.7410072088241577.
[I 2022-04-

[I 2022-04-02 19:31:29,571] Trial 26 finished with value: 0.7410072088241577 and parameters: {'num_layer': 3, 'filters_0': 32, 'kernel_size_0': 14, 'filters_1': 4, 'kernel_size_1': 7, 'filters_2': 32, 'kernel_size_2': 13, 'learning_rate': 4.473265278831879e-05}. Best is trial 25 with value: 0.769784152507782.
[I 2022-04-02 19:38:47,657] Trial 27 finished with value: 0.7338129281997681 and parameters: {'num_layer': 3, 'filters_0': 32, 'kernel_size_0': 15, 'filters_1': 256, 'kernel_size_1': 10, 'filters_2': 32, 'kernel_size_2': 9, 'learning_rate': 1.1011766973699417e-05}. Best is trial 25 with value: 0.769784152507782.
[I 2022-04-02 19:40:59,192] Trial 28 finished with value: 0.7410072088241577 and parameters: {'num_layer': 3, 'filters_0': 32, 'kernel_size_0': 13, 'filters_1': 4, 'kernel_size_1': 9, 'filters_2': 64, 'kernel_size_2': 13, 'learning_rate': 2.8175571186899747e-05}. Best is trial 25 with value: 0.769784152507782.
[I 2022-04-02 19:42:59,141] Trial 29 finished with value: 0.733

[I 2022-04-02 22:08:41,809] Trial 52 finished with value: 0.7194244861602783 and parameters: {'num_layer': 3, 'filters_0': 32, 'kernel_size_0': 14, 'filters_1': 256, 'kernel_size_1': 15, 'filters_2': 32, 'kernel_size_2': 12, 'learning_rate': 1.0446779833549042e-05}. Best is trial 47 with value: 0.7769784331321716.
[I 2022-04-02 22:16:21,665] Trial 53 finished with value: 0.7194244861602783 and parameters: {'num_layer': 3, 'filters_0': 8, 'kernel_size_0': 15, 'filters_1': 256, 'kernel_size_1': 11, 'filters_2': 32, 'kernel_size_2': 14, 'learning_rate': 2.06697291971898e-05}. Best is trial 47 with value: 0.7769784331321716.
[I 2022-04-02 22:23:58,206] Trial 54 finished with value: 0.7410072088241577 and parameters: {'num_layer': 3, 'filters_0': 16, 'kernel_size_0': 12, 'filters_1': 256, 'kernel_size_1': 14, 'filters_2': 4, 'kernel_size_2': 10, 'learning_rate': 4.8391941123882796e-05}. Best is trial 47 with value: 0.7769784331321716.
[I 2022-04-02 22:35:22,387] Trial 55 finished with value

[I 2022-04-03 00:38:18,887] Trial 78 finished with value: 0.6834532618522644 and parameters: {'num_layer': 3, 'filters_0': 32, 'kernel_size_0': 14, 'filters_1': 128, 'kernel_size_1': 8, 'filters_2': 128, 'kernel_size_2': 12, 'learning_rate': 5.552519222858159e-05}. Best is trial 65 with value: 0.7841726541519165.
[I 2022-04-03 00:46:38,399] Trial 79 finished with value: 0.7338129281997681 and parameters: {'num_layer': 3, 'filters_0': 256, 'kernel_size_0': 13, 'filters_1': 16, 'kernel_size_1': 11, 'filters_2': 256, 'kernel_size_2': 10, 'learning_rate': 1.2318004782574225e-05}. Best is trial 65 with value: 0.7841726541519165.
[I 2022-04-03 00:48:32,021] Trial 80 finished with value: 0.7410072088241577 and parameters: {'num_layer': 3, 'filters_0': 32, 'kernel_size_0': 15, 'filters_1': 4, 'kernel_size_1': 3, 'filters_2': 32, 'kernel_size_2': 12, 'learning_rate': 1.9696139905244937e-05}. Best is trial 65 with value: 0.7841726541519165.
[I 2022-04-03 00:50:51,008] Trial 81 finished with valu

Number of finished trials: 100
Best trial:
  Value: 0.7841726541519165
  Params: 
    num_layer: 3
    filters_0: 32
    kernel_size_0: 14
    filters_1: 4
    kernel_size_1: 11
    filters_2: 32
    kernel_size_2: 11
    learning_rate: 0.0002965967934294565


In [24]:
df_results = study.trials_dataframe()
df_results.to_csv('DistalPhalanxOutlineAgeGroup_df_optuna_results_nodropout_epoch500_trial100_020422.csv')


In [25]:
plt.plot([t.value for t in study.trials])

In [26]:
optuna.visualization.plot_contour(study)

In [27]:
optuna.visualization.plot_param_importances(study)

In [28]:
study.best_params

{'num_layer': 3,
 'filters_0': 32,
 'kernel_size_0': 14,
 'filters_1': 4,
 'kernel_size_1': 11,
 'filters_2': 32,
 'kernel_size_2': 11,
 'learning_rate': 0.0002965967934294565}

In [29]:
study.best_trial

FrozenTrial(number=65, values=[0.7841726541519165], datetime_start=datetime.datetime(2022, 4, 3, 0, 2, 27, 44316), datetime_complete=datetime.datetime(2022, 4, 3, 0, 4, 32, 902042), params={'num_layer': 3, 'filters_0': 32, 'kernel_size_0': 14, 'filters_1': 4, 'kernel_size_1': 11, 'filters_2': 32, 'kernel_size_2': 11, 'learning_rate': 0.0002965967934294565}, distributions={'num_layer': IntUniformDistribution(high=3, low=1, step=1), 'filters_0': CategoricalDistribution(choices=(4, 8, 16, 32, 64, 128, 256)), 'kernel_size_0': IntUniformDistribution(high=15, low=3, step=1), 'filters_1': CategoricalDistribution(choices=(4, 8, 16, 32, 64, 128, 256)), 'kernel_size_1': IntUniformDistribution(high=15, low=3, step=1), 'filters_2': CategoricalDistribution(choices=(4, 8, 16, 32, 64, 128, 256)), 'kernel_size_2': IntUniformDistribution(high=15, low=3, step=1), 'learning_rate': LogUniformDistribution(high=0.1, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=65, state=Tria

In [46]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_drop_out_0,params_drop_out_1,params_drop_out_2,params_filters_0,params_filters_1,params_filters_2,params_kernel_size_0,params_kernel_size_1,params_kernel_size_2,params_learning_rate,params_num_layer,state
0,0,0.561151,2022-03-25 15:07:37.224764,2022-03-25 15:10:56.458671,0 days 00:03:19.233907,0.35,0.50,0.10,128,64.0,NaN,5,8.0,NaN,0.012791,2,COMPLETE
1,1,0.719424,2022-03-25 15:10:56.460665,2022-03-25 15:14:08.117601,0 days 00:03:11.656936,0.50,0.25,0.10,128,64.0,NaN,3,8.0,NaN,0.000086,2,COMPLETE
2,2,0.503597,2022-03-25 15:14:08.120594,2022-03-25 15:16:08.545873,0 days 00:02:00.425279,0.45,0.35,0.35,32,64.0,64.0,5,5.0,3.0,0.000053,3,COMPLETE
3,3,0.467626,2022-03-25 15:16:08.546871,2022-03-25 15:17:10.725001,0 days 00:01:02.178130,0.35,0.15,0.10,64,NaN,NaN,8,NaN,NaN,0.000020,1,COMPLETE
4,4,0.640288,2022-03-25 15:17:10.726995,2022-03-25 15:18:48.337362,0 days 00:01:37.610367,0.30,0.35,0.20,32,64.0,NaN,8,5.0,NaN,0.000283,2,COMPLETE
5,5,0.467626,2022-03-25 15:18:48.339356,2022-03-25 15:19:36.744271,0 days 00:00:48.404915,0.40,0.30,0.50,32,NaN,NaN,5,NaN,NaN,0.005366,1,COMPLETE
6,6,0.697842,2022-03-25 15:19:36.745267,2022-03-25 15:21:01.261934,0 days 00:01:24.516667,0.40,0.40,0.05,32,64.0,NaN,3,3.0,NaN,0.000046,2,COMPLETE
7,7,0.561151,2022-03-25 15:21:01.263905,2022-03-25 15:30:16.685009,0 days 00:09:15.421104,0.15,0.25,0.30,32,256.0,256.0,3,3.0,5.0,0.000678,3,COMPLETE
8,8,0.755396,2022-03-25 15:30:16.688000,2022-03-25 15:42:27.447680,0 days 00:12:10.759680,0.30,0.10,0.30,256,256.0,256.0,5,5.0,3.0,0.000062,3,COMPLETE
9,9,0.712230,2022-03-25 15:42:27.449677,2022-03-25 15:45:14.616938,0 days 00:02:47.167261,0.10,0.40,0.05,128,64.0,32.0,8,3.0,5.0,0.000021,3,COMPLETE


In [30]:
optuna.visualization.plot_optimization_history(study)

In [14]:
optuna.visualization.plot_parallel_coordinate(study)

ValueError: all keys need to be the same shape

In [12]:
optuna.visualization.plot_slice(study)

# 測試1

In [71]:
# class=4，but BME's category = 3
# loss="sparse_categorical_crossentropy" 

def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    # ------------------------------
    # read data
    archive_name = "UCRArchive_2018"
    dataset_name = 'BME'
    root_dir = 'D:/FH_Dortmund/Projekt/Projektarbeit_2/time_series/dl-4-tsc-master/dataset'
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)
    
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_valid = datasets_dict[dataset_name][2]
    y_valid = datasets_dict[dataset_name][3]
    
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)).astype("float32") / 255
    x_valid = x_valid.reshape((x_valid.shape[0], x_valid.shape[1], 1)).astype("float32") / 255
#    y_train = y_train[:N_TRAIN_EXAMPLES].astype(int)
#    y_valid = y_valid[:N_VALID_EXAMPLES].astype(int)

    input_shape = x_train.shape[1:] 
    
    num_layer = trial.suggest_int("num_layer", 2, 3)
    
    model = Sequential()
    model.add(
        Conv1D(
            filters=trial.suggest_categorical("filters", [32, 64, 128, 256]),
            kernel_size=trial.suggest_categorical("kernel_size", [3, 5, 8]),
            activation = "relu",
            padding="same",
            input_shape=input_shape,
        )
    )
    model.add(BatchNormalization())
    
    for i in range(1,num_layer):
        model.add(
            Conv1D(
                filters=trial.suggest_categorical("filters", [32, 64, 128, 256]), 
                kernel_size=trial.suggest_categorical("kernel_size", [3, 5, 8]), 
                padding="same", 
                activation="relu"))
        model.add(BatchNormalization())
        
   
    
    model.add(GlobalAveragePooling1D())
    
    model.add(Flatten())
    model.add(Dense(CLASSES, activation="softmax"))

    tf.keras.callbacks.History()
    
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=RMSprop(learning_rate=learning_rate),
        metrics=["accuracy"],
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

In [72]:
if __name__ == "__main__":
    warnings.warn(
        "Recent Keras release (2.4.0) simply redirects all APIs "
        "in the standalone keras package to point to tf.keras. "
        "There is now only one Keras: tf.keras. "
        "There may be some breaking changes for some workflows by upgrading to keras 2.4.0. "
        "Test before upgrading. "
        "REF:https://github.com/keras-team/keras/releases/tag/2.4.0"
    )
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

D:\TMP/ipykernel_3504/2353197814.py:2: UserWarning: Recent Keras release (2.4.0) simply redirects all APIs in the standalone keras package to point to tf.keras. There is now only one Keras: tf.keras. There may be some breaking changes for some workflows by upgrading to keras 2.4.0. Test before upgrading. REF:https://github.com/keras-team/keras/releases/tag/2.4.0
  warnings.warn(
[I 2022-03-22 15:59:01,701] A new study created in memory with name: no-name-0a713a39-cca8-4e61-9a86-4b140592540c
[I 2022-03-22 15:59:04,079] Trial 0 finished with value: 0.3333333432674408 and parameters: {'num_layer': 3, 'filters': 64, 'kernel_size': 5, 'learning_rate': 0.00019573124104154576}. Best is trial 0 with value: 0.3333333432674408.
[I 2022-03-22 15:59:07,089] Trial 1 finished with value: 0.3333333432674408 and parameters: {'num_layer': 3, 'filters': 128, 'kernel_size': 3, 'learning_rate': 6.572428821657196e-05}. Best is trial 0 with value: 0.3333333432674408.


KeyboardInterrupt: 